In [17]:
from transformers import AutoModelForSequenceClassification, BertTokenizer
import torch
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score

In [18]:
# 读取测试数据
# test_df = pd.read_csv("./data/test/NSL-KDD-10000.csv")
# test_df = pd.read_csv("./data/test/CICIDS2018-10000.csv")
# test_df = pd.read_csv("./data/test/KDD99-10000.csv")
test_df = pd.read_csv("./data/test/UNSW_NB15-10000.csv")
# test_df = pd.read_csv("./data/test/X-IIoTID-10000.csv")
texts = test_df["flow"].tolist()

In [19]:
# 使用预训练的 BERT 模型（不微调）
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = BertTokenizer.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")

In [21]:
# 遍历数据，找到最长文本的长度
text_lengths = [len(tokenizer(text)["input_ids"]) for text in texts]
max_length = max(text_lengths)
max_length = min(max_length, 500)
max_length

163

In [22]:
def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")

In [23]:
# 关闭梯度计算，加速推理
model.eval()
torch.set_grad_enabled(False)
device = torch.device("cuda")
model.to(device)
# 进度条 & 批量推理
batch_size = 16  # 设置合适的 batch size
predictions = []
for i in tqdm(range(0, len(texts), batch_size), desc="预测进度", unit="batch"):
    batch_texts = texts[i : i + batch_size]
    inputs = tokenize_function(batch_texts)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    # 运行模型预测
    outputs = model(**inputs)
    batch_predictions = torch.argmax(outputs.logits, dim=-1).tolist()
    predictions.extend(batch_predictions)

预测进度: 100%|██████████| 625/625 [00:37<00:00, 16.83batch/s]


In [24]:
# 计算指标
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
if "class" in test_df.columns:
    true_labels = test_df["class"].tolist()
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='binary')  # 二分类使用binary
    recall = recall_score(true_labels, predictions, average='binary')
    f1 = f1_score(true_labels, predictions, average='binary')
    print(f"\n模型在测试集上的指标:")
    print(f"准确率 (Accuracy): {accuracy:.4f}")
    print(f"精确率 (Precision): {precision:.4f}")
    print(f"召回率 (Recall): {recall:.4f}")
    print(f"F1 分数 (F1-score): {f1:.4f}")


模型在测试集上的指标:
准确率 (Accuracy): 0.0002
精确率 (Precision): 0.0000
召回率 (Recall): 0.0000
F1 分数 (F1-score): 0.0000


c:\Users\njust\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
